In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Predictors of mental health illness** <br>
2014년 설문조사. 기술 사업장에서의 정신건강에 대한 태도, 정신건강 장애 빈도를 측정하는 설문<br>

원본 데이터 셋(캐글) : https://www.kaggle.com/datasets/osmi/mental-health-in-tech-survey<br>
참고 코드 : https://www.kaggle.com/code/kairosart/machine-learning-for-mental-health-1<br>

### 01. 라이브러리 불러오기

In [1]:
import pandas as pd  # 데이터 파일 작업
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, binarize #라벨 인코더
from sklearn.model_selection import train_test_split         #데이터 나누기
from sklearn.datasets import make_classification

In [3]:
from scipy import stats
from scipy.stats import randint

In [5]:
%%time
surv = pd.read_csv("../input/mental-health-in-tech-survey/survey.csv")

In [6]:
surv.shape

In [7]:
surv.head()

In [8]:
surv.tail()

In [9]:
surv.info()

In [10]:
surv.columns

### 02. 데이터 불러오기 / 확인하기
* 1259개행과 27개열(column)의 데이터로 구성되어 있다.
  * 'Timestamp' : 타임스탬프 2014-08-27 ~ 2016-02-01
  * 'Age' : 나이
  * 'Gender' : 성별
  * 'Country' : 국가
  * 'state' : 미국 어느 주에 살고 있는지?
  * 'self_employed' : 자영업자인지?
  * 'family_history' : 정신질환 가족력
  * 'treatment' : 정신질환 치료력
  * 'work_interfere' : 정신 건강 문제가 업무에 방해가 되는지?
  * 'no_employees' : 직원 / 동료의 수
  * 'remote_work' : 시간의 50% 이상 원격(사무실 외부)으로 작업합니까?
  * 'tech_company' : 기술 사업장인지?
  * 'benefits' : 정신건강 혜택이 있는지?
  * 'care_options' : 정신건강 관리 옵션이 있는지?
  * 'wellness_program' : 건강 프로그램에 대해 논의한 적이 있습니까?
  * 'seek_help' : 정신건강 문제와 도움을 구할 수 있는 방법에 대해 리소스를 지공하는지?
  * 'anonymity' : 익명성이 보호되는지?
  * 'leave' : 정신건강상의 병가 가능한지?
  * 'mental_health_consequence' : 정신건강 결과를 open 하는 것이 부정적이라 생각하는지?
  * 'phys_health_consequence' : 신체건강 결과를 open하는 것이 부정적이라 생각하는지?
  * 'coworkers' : 동료와 정신건강 문제를 논의할 의향이 있는지?
  * 'supervisor' : 직속상사와 정신건강 문제를 논의할 의향이 있는지?
  * 'mental_health_interview' : 인터뷰에서 정신건강 문제를 제기할지?
  * 'phys_health_interview'; 인터뷰에서 신체건강 문제를 제기할지?
  * 'mental_vs_physical' : 고용주가 정신건강을 신체건강만큼 중요하게 생각하는가?
  * 'obs_consequence' : 직장에서 정신건강문제의 부정적인 결과를 듣거나 관찰한 적이 있는가?
  * 'comments' : 추가 메모나 의견
  
* 결측치
  * 미국 거주 -> 거주하는 주(state) 확인 : 744개의 응답.
  * 자영업자 여부(self_employed) -> 1241개의 응답.
  * 정신건강 문제가 업무에 방해가 되는지(work_interfere) -> 995개의 응답.
  * 기타 추가적인 메모나 의견(comments) -> 164개의 응답.

* Inspiration 를 보니 다음을 제시하고 있다.
  * 정신 질환 빈도나 정신 건강 태도가 지리적 위치에 따라 어떻게 다른지
  * 직장에서 특정 태도를 예측할 수 있는 변수는 무엇인지


### **->데이터 다양하게 확인해보고, 시각화 해보기를 진행하고자 한다.**

In [11]:
surv.describe()

* describe()를 찍어보면 수치형 변수인 Age에 대한 값만 확인할 수 있다.

### 03. 데이터 정제(Data Cleaning), 정규화(Normalizing)
* 모델을 훈련할 수 있는 데이터를 확인하기 위해서 결측치(NaN), 이상치(outlier)를 확인하는 것.
* 결측값 isnull().sum()
* 평균값으로 NaN 대체 fillna(mean())
* NaN 제거 dropna()
* 데이터를 정제한 후 데이터 범위의 숫자 열 값을 변경하여 값 범위의 차이를 수정하지 않고 공통 척도를 적용하는 것.